In [6]:
"""A naive little Merkel tree implementation.
Which hash to use depends on the purpose.
For some applications, it will suffice with a simple CRC algorithm.
The input is a textfile, where each line is to be a block.
"""

import sys
import math
from bitstring import Bits
from hashlib import sha256
block = list()
m3_lol = list()
#
# The Merkle tree is stored in a list-of-list.
# Each list represent a level
# The label for leaf nodes are the level+parent-node label.
# The hash will include the hashes of the child + the level.
#
merkle3 = list()

def build_merkle_3(block : list) -> list:
"""Build a Merkle tree"""
    level = math.ceil(math.log2(len(block)))
    m3 = [[i] for i in range(level+1)]
# compute leaf-nodes
    m3[level] = list()
    m3[0] = (12,b'')
    for i in range(2**level):
        label = Bits(uint=i,length=level).bin
        if i<len(block):
            m3[level].append((label,sha256(bytes(block[i],"utf-8")).digest()))
        else:
            m3[level].append((label,b''))
            
    while level>1:
        level -= 1
        m3[level] = list()
        for i in range(2**level):
            label = Bits(uint=i,length=level).bin
            hash = sha256()
            child_0 = int(label+'0',base=2)
            if (child_0 <= len(block)):
                child_0_label = m3[level+1][child_0][0]
                child_0_hash = m3[level+1][child_0][1]
                hash.update(bytes(child_0_label,"utf-8"))
                hash.update(child_0_hash)

            child_1 = int(label+'1',base=2)
            if (child_1 <= len(block)):
                child_1_label = m3[level+1][child_1][0]
                child_1_hash = m3[level+1][child_1][1]
                hash.update(bytes(child_0_label,"utf-8"))
                hash.update(child_0_hash)
        #print("Level:",level, child_0, child_1,flush=True)
            m3[level].append((label,hash.digest()))
    tophash = sha256()
    tophash.update(bytes(m3[1][0][0],"utf-8"))
    tophash.update(m3[1][0][1])
    tophash.update(bytes(m3[1][1][0],"utf-8"))
    tophash.update(m3[1][1][1])
    m3[0] = (12,tophash.digest())
    return m3

print("\n*** A naive 'Merkle 3, with lines from RFC 9162 as blocks.\n")

if __name__ == "__main__":
    fname = "rfc9162.txt"
    print("Reading the file.")
    try:
        with open(fname, 'r') as source:
            block = source.readlines()
    except:
        print("Couldn't read '"+fname+"'.")
        sys.exit(1)
        print("No.of lines : ",len(block))
        merkle3 = build_merkle_3(block)

IndentationError: expected an indented block after function definition on line 21 (1767652481.py, line 22)